In [ ]:
%reload_ext setup

In [ ]:
from igent.models import get_model_client
from autogen_core.models import UserMessage

openai_client = await get_model_client("openai")
result = await openai_client.create([UserMessage(content="What is the capital of France?", source="user")])  # type: ignore
print(result.content)

In [ ]:
from igent.models import get_model_client
from autogen_core.models import UserMessage

openai_client = await get_model_client("azure")
result = await openai_client.create([UserMessage(content="What is the capital of France?", source="user")])  # type: ignore
print(result.content)

## Azure call

In [ ]:
import requests
import json
import os

# Replace these with your actual Azure AI Foundry deployment details
ENDPOINT_URL = "https://aiservices-pom-poc-westeu-001.services.ai.azure.com/models/chat/completions?api-version=2024-05-01-preview"

API_KEY = os.getenv("AZUREAI_API_KEY")

# Define the headers
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

# Define the payload (simple test message)
payload = {
    "model": "DeepSeek-V3",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, this is a test call from Azure AI. How are you?"},
        {"role": "user", "content": "How can I make a call to you via AutoGen?"}
    ],
    "stream": False
}

# Make the API call
response = requests.post(ENDPOINT_URL, headers=headers, data=json.dumps(payload))

# Check the response
if response.status_code == 200:
    result = response.json()
    print("Response from DeepSeek-R1:")
    print(result["choices"][0]["message"]["content"])
else:
    print(f"Error: {response.status_code} - {response.text}")

## AzureAIClientCompletion

In [ ]:
import os
from azure.core.credentials import AzureKeyCredential
from autogen_ext.models.azure import AzureAIChatCompletionClient
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage, SystemMessage, ModelInfo

# Azure AI Foundry deployment details
ENDPOINT_URL = "https://aiservices-pom-poc-westeu-001.services.ai.azure.com/models/"
API_VERSION = "2024-05-01-preview"
API_KEY = os.getenv("AZUREAI_API_KEY")

# Initialize the AzureAIChatCompletionClient with credential
client = AzureAIChatCompletionClient(
    model="DeepSeek-V3",
    endpoint=ENDPOINT_URL,
    credential=AzureKeyCredential(API_KEY),
    api_version=API_VERSION,
    model_info=ModelInfo(
        name="DeepSeek-V3",
        vision=False,
        function_calling=True,
        json_output=False,
        family="r1",  # Assuming "r1" is correct for your DeepSeek-V3 deployment
    )
)

result = await client.create([UserMessage(content="Just tell a joke?", source="user")])
print(result.content)